## IMPORTS and UTILS

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import entropy
from math import sqrt


from robust_evaluation_tools.robust_utils import get_complete_combination
from robust_evaluation_tools.robust_outlier_detection import find_outliers, get_metrics
from robust_evaluation_tools.synthectic_sites_generations import generate_sites
from robust_evaluation_tools.robust_analysis import calculate_precision_by_bundle
from clinical_combat.harmonization.QuickCombat import QuickCombat



MAINFOLDER = "PRECISION_TEST"
SYNTHETIC_SITES = f"{MAINFOLDER}/SYNTHETIC_SITES"

ANALYSIS_FOLDER = "ANALYSIS"

In [2]:
# Fonction pour calculer la divergence KL
def kl_divergence(p, q):
    return entropy(p, q)

## EXECUTOR

In [ ]:
# combination = get_complete_combination(os.path.join('DONNES','CamCAN'), f'CamCAN.*.raw.csv.gz',False, True)
# combination['metric_bundle'] = combination['metric'] + "_" + combination['bundle']
# combination.to_csv(os.path.join('DONNES', f'CamCAN_combination_all_metrics.csv.gz'), index=False)

In [3]:
SYNTHETIC_SITES_VERSION = "v1"
include_CamCAN = True
sites_dir = os.path.join(SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION)
os.makedirs(SYNTHETIC_SITES, exist_ok=True)
raw_directory = os.path.join('DONNES', 'COMPILATIONS')
# for disease in sorted(os.listdir(raw_directory)):
#     if os.path.isdir(os.path.join(raw_directory, disease)):
#         print('--------------------', disease, '-----------------------')
#         combination = get_complete_combination(os.path.join(raw_directory, disease), f'{disease}_combination*.csv.gz', include_CamCAN, False)
#         combination['metric_bundle'] = combination['metric'] + "_" + combination['bundle']
#         if include_CamCAN:
#             combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'), index=False)
#         else:
#             combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics.csv.gz'), index=False)

path = os.path.join('DONNES', 'adni_combination_all_metrics.csv.gz')

print('do')


#sample_sizes = [30, 50, 100, 150, 200, 300]  # Différentes tailles d'échantillon
#disease_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7]  # Différents pourcentages de malades
sample_sizes = [200]  # Différentes tailles d'échantillon
disease_ratios = [0.3]  # Différents pourcentages de malades
num_tests = 30  # Nombre de tests à effectuer pour chaque combinaison

#generate_sites(sample_sizes, disease_ratios, num_tests, sites_dir, path, disease)




do


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
HC_LABEL = 'HC'

# Plot distribution for each bundle
for disease in [d for d in sorted(os.listdir(raw_directory)) if os.path.isdir(os.path.join(raw_directory, d))]:
    combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
    metric_bundles = combination['metric_bundle'].unique()
    for metric_bundle in metric_bundles:
        subset_all = combination[combination['metric_bundle'] == metric_bundle]
        subset = subset_all[subset_all['old_site'] != 'CamCAN']
        bundle = subset_all.bundle.unique()[0]
        metric = subset_all.metric.unique()[0]
        camCan_subset = subset_all[subset_all['old_site'] == 'CamCAN']
        
        # Scatter plot with mean
        plt.figure(figsize=(10, 6))
        plt.scatter(subset[subset['disease'] == HC_LABEL]['age'], subset[subset['disease'] == HC_LABEL]['mean'], color='blue', label='HC', alpha=0.5)
        plt.scatter(camCan_subset['age'], camCan_subset['mean'], color='green', label='CamCAN', alpha=0.5)
        plt.scatter(subset[subset['disease'] != HC_LABEL]['age'], subset[subset['disease'] != HC_LABEL]['mean'], color='red', label=f'{disease}', alpha=0.5)
        
        plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean)')
        plt.xlabel('Age')
        plt.ylabel('Mean')
        plt.legend()
        plt.show()
        
        # Scatter plot with mean_no_cov
        plt.figure(figsize=(10, 6))
        plt.scatter(subset[subset['disease'] == HC_LABEL]['age'], subset[subset['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', alpha=0.5)
        plt.scatter(camCan_subset['age'], camCan_subset['mean_no_cov'], color='green', label='CamCAN', alpha=0.5)
        plt.scatter(subset[subset['disease'] != HC_LABEL]['age'], subset[subset['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', alpha=0.5)
        
        plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean_no_cov)')
        plt.xlabel('Age')
        plt.ylabel('Mean_no_cov')
        plt.legend()
        plt.show()

In [5]:
import seaborn as sns

import matplotlib.pyplot as plt

# Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
HC_LABEL = 'HC'
DISTRIBUTION_FOLDER = 'COMPILATION_DISTRIBUTION3'

# Plot distribution for each bundle
for disease in [d for d in sorted(os.listdir(raw_directory)) if os.path.isdir(os.path.join(raw_directory, d))]:
    combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
    metric_bundles = combination['metric_bundle'].unique()
    print('--------------------', disease, '-----------------------')
    for metric_bundle in metric_bundles:
        subset_all = combination[combination['metric_bundle'] == metric_bundle]
        subset = subset_all[subset_all['old_site'] != 'CamCAN']
        bundle = subset_all.bundle.unique()[0]
        metric = subset_all.metric.unique()[0]
        camCan_subset = subset_all[subset_all['old_site'] == 'CamCAN']

        plt.figure(figsize=(10, 6))
        sns.kdeplot(subset[subset['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', fill=True)
        sns.kdeplot(camCan_subset['mean_no_cov'], color='green', label=f'CamCAN', fill=True)
        sns.kdeplot(subset[subset['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', fill=True)
        plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle}')
        plt.xlabel('mean_no_cov')
        plt.ylabel('Frequency')
        plt.legend()
        output_dir = os.path.join(DISTRIBUTION_FOLDER, disease, metric)
        os.makedirs(output_dir, exist_ok=True)

        # Save the plot
        output_path = os.path.join(output_dir, f'{metric_bundle}.png')
        plt.savefig(output_path)
        plt.close()
        

/tmp/ipykernel_717584/350362291.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- AD -----------------------


/tmp/ipykernel_717584/350362291.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- ADHD -----------------------


/tmp/ipykernel_717584/350362291.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


...

-------------------- MCI -----------------------


/tmp/ipykernel_717584/350362291.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- SCHZ -----------------------


/tmp/ipykernel_717584/350362291.py:11: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- TBI -----------------------


In [4]:
# Assuming 'HC' is a specific value in the 'disease' column that indicates healthy controls
HC_LABEL = 'HC'

# Initialisation du DataFrame de résultat
results = []

# Parcourir chaque dossier de maladie
for disease in [d for d in sorted(os.listdir(raw_directory)) if os.path.isdir(os.path.join(raw_directory, d))]:
    combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))
    metric_bundles = combination['metric_bundle'].unique()
    print('--------------------', disease, '-----------------------')
    for metric_bundle in metric_bundles:
        subset_all = combination[combination['metric_bundle'] == metric_bundle]
        bundle = subset_all.bundle.unique()[0]
        metric = subset_all.metric.unique()[0]

        # Subsets pour HC et non-HC
        hc_data = subset_all[subset_all['disease'] == HC_LABEL]['mean_no_cov'].dropna()
        non_hc_data = subset_all[subset_all['disease'] != HC_LABEL]['mean_no_cov'].dropna()

        # Vérification que les deux distributions ne sont pas vides
        if len(hc_data) > 1 and len(non_hc_data) > 1:
            # Créer des histogrammes pour normaliser les distributions
            hc_hist, bins = np.histogram(hc_data, bins=50, density=True)
            non_hc_hist, _ = np.histogram(non_hc_data, bins=bins, density=True)

            # Calcul des métriques
            mean_hc = np.mean(hc_data)
            mean_non_hc = np.mean(non_hc_data)
            std_hc = np.std(hc_data)
            std_non_hc = np.std(non_hc_data)

            bhatt_dist = QuickCombat.bhattacharyya_distance(hc_data,non_hc_data)
            kl_div = kl_divergence(hc_hist + 1e-10, non_hc_hist + 1e-10)  # Ajout d'un epsilon pour éviter les divisions par zéro
            euclidean_dist = abs(mean_hc - mean_non_hc)
            mahalanobis_dist = abs(mean_hc - mean_non_hc) / sqrt((std_hc**2 + std_non_hc**2) / 2)

            # Ajouter les résultats dans une liste
            results.append({
                'disease': disease,
                'metric_bundle': metric_bundle,
                'bundle': bundle,
                'metric': metric,
                'bhattacharyya_distance': bhatt_dist,
                'kl_divergence': kl_div,
                'euclidean_distance': euclidean_dist,
                'mahalanobis_distance': mahalanobis_dist
            })
        

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Sauvegarder le DataFrame dans un fichier CSV
output_path = 'DISTRIBUTION_RESULTS/distance_metrics_results.csv'
results_df.to_csv(output_path, index=False)

print(f"Résultats sauvegardés dans {output_path}")

/tmp/ipykernel_718908/1790064209.py:9: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- AD -----------------------


/tmp/ipykernel_718908/1790064209.py:9: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- ADHD -----------------------


/tmp/ipykernel_718908/1790064209.py:9: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


...

-------------------- MCI -----------------------


/tmp/ipykernel_718908/1790064209.py:9: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- SCHZ -----------------------


/tmp/ipykernel_718908/1790064209.py:9: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  combination = pd.read_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'))


-------------------- TBI -----------------------
Résultats sauvegardés dans DISTRIBUTION_RESULTS/distance_metrics_results.csv


In [5]:
# Initialisation du DataFrame de résultat pour les valeurs maximales
max_results = []

# Parcourir chaque maladie
for disease in results_df['disease'].unique():
    disease_subset = results_df[results_df['disease'] == disease]
    
    # Identifier les metric_bundles avec les valeurs maximales pour chaque mesure de distance
    max_bhatt = disease_subset.loc[disease_subset['bhattacharyya_distance'].idxmax()]
    max_kl = disease_subset.loc[disease_subset['kl_divergence'].idxmax()]
    max_euclidean = disease_subset.loc[disease_subset['euclidean_distance'].idxmax()]
    max_mahalanobis = disease_subset.loc[disease_subset['mahalanobis_distance'].idxmax()]
    
    # Ajouter les résultats dans une liste
    max_results.append({
        'disease': disease,
        'metric_bundle_max_bhatt': max_bhatt['metric_bundle'],
        'bhattacharyya_distance': max_bhatt['bhattacharyya_distance'],
        'metric_bundle_max_kl': max_kl['metric_bundle'],
        'kl_divergence': max_kl['kl_divergence'],
        'metric_bundle_max_euclidean': max_euclidean['metric_bundle'],
        'euclidean_distance': max_euclidean['euclidean_distance'],
        'metric_bundle_max_mahalanobis': max_mahalanobis['metric_bundle'],
        'mahalanobis_distance': max_mahalanobis['mahalanobis_distance']
    })

# Convertir les résultats en DataFrame
max_results_df = pd.DataFrame(max_results)

# Afficher les résultats
print(max_results_df)

  disease metric_bundle_max_bhatt  bhattacharyya_distance  \
0      AD             fw_mni_UF_L                0.289255   
1    ADHD            fa_mni_ICP_L                0.463382   
2     BIP            fw_mni_ICP_R                0.456271   
3     MCI            fw_mni_ICP_R                0.142345   
4    SCHZ           fat_mni_CST_R                0.308303   
5     TBI           afd_mni_SLF_R                0.248530   

  metric_bundle_max_kl  kl_divergence metric_bundle_max_euclidean  \
0         rd_mni_CST_L       6.518791                fw_mni_F_L_R   
1          mdt_mni_C_R      12.282830               afd_mni_ICP_L   
2         adt_mni_AF_L      11.237278               fat_mni_F_L_R   
3        adt_mni_PPT_L       1.412855                fw_mni_F_L_R   
4         adt_mni_AF_R       6.612277              fat_mni_IFOF_R   
5         afd_mni_OR_L       3.654554               afd_mni_VOF_L   

   euclidean_distance metric_bundle_max_mahalanobis  mahalanobis_distance  
0           